# Simple Approaches to Multi-Label Classification: Motivation

### Random Training Dataset

1. This is a version where the labels of the training dataset are randomly set, in an attempt to show that better classifiers still perform better even for noisy (high-inconsistently labeled) dataset, but not for randomly assigned dataset.
2. This program is modified from https://github.com/nkartik94/Multi-Label-Text-Classification on 2019/11/18 by Yuen-Hsien Tseng

## 1. EDA: Exploratory Data Analysis

In [1]:
import os, sys, time
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
#printmd('**bold**')

In [3]:
data_path  = "mlabel_corpora/JokeMotivation.txt"
train_path = "mlabel_corpora/JokeMotivation_train_random.txt"
test_path  = "mlabel_corpora/JokeMotivation_test.txt"

In [4]:
# set global variables: df
df = pd.read_csv(data_path, delimiter="\t")
train_random = pd.read_csv(train_path, delimiter="\t")
test = pd.read_csv(test_path, delimiter="\t")
print(df.shape, train_random.shape, test.shape) # same as data_raw.shape in Jupyter

(3365, 9) (1691, 9) (1674, 9)


### 1.1. Checking for missing values

In [5]:
missing_values_check = df.isnull().sum()
print(missing_values_check)

ID                  0
Title               0
Content             0
Affinity            0
Self_Improvement    0
Attack              0
Self_Depression     0
Taboo               0
Others              0
dtype: int64


### 1.2. Calculating number of jokes under each label

In [6]:
# Jokes with no label are considered to be clean jokes.
# Creating seperate column in dataframe to identify clean jokes.
# We use axis=1 to count row-wise and axis=0 to count column wise
def print_empty_label(df, s):
    rowSums = df.iloc[:,3:].sum(axis=1)
    #print(rowSums.shape)
    #print(rowSums.head())
    clean_comments_count = (rowSums==0).sum(axis=0)

    print(f"Total number of {s} jokes = ",len(df))
    print(f"Number of clean jokes in {s}= ",clean_comments_count)
    print(f"Number of {s} jokes with labels =",(len(df)-clean_comments_count))
    print()

In [7]:
print_empty_label(df, 'all')
print_empty_label(train_random, 'train_random')
print_empty_label(test, 'test')

Total number of all jokes =  3365
Number of clean jokes in all=  6
Number of all jokes with labels = 3359

Total number of train_random jokes =  1691
Number of clean jokes in train_random=  5
Number of train_random jokes with labels = 1686

Total number of test jokes =  1674
Number of clean jokes in test=  1
Number of test jokes with labels = 1673



In [8]:
# set global variables: categories
categories = list(df.columns.values)
print(categories)
categories = categories[3:]
print(categories)

['ID', 'Title', 'Content', 'Affinity', 'Self_Improvement', 'Attack', 'Self_Depression', 'Taboo', 'Others']
['Affinity', 'Self_Improvement', 'Attack', 'Self_Depression', 'Taboo', 'Others']


In [9]:
# Calculating number of jokes in each category
def print_category_count(df, categories):
    counts = []
    for category in categories:
        counts.append((category, df[category].sum()))
    df_stats = pd.DataFrame(counts, columns=['category', 'number of jokes'])
    print(df_stats)
    print()

In [10]:
print_category_count(df, categories)
print_category_count(train_random, categories)
print_category_count(test, categories)

           category  number of jokes
0          Affinity               92
1  Self_Improvement               43
2            Attack              409
3   Self_Depression               64
4             Taboo              584
5            Others             2226

           category  number of jokes
0          Affinity               73
1  Self_Improvement               27
2            Attack              266
3   Self_Depression               47
4             Taboo              355
5            Others              972

           category  number of jokes
0          Affinity               19
1  Self_Improvement               16
2            Attack              143
3   Self_Depression               17
4             Taboo              229
5            Others             1254



In [11]:
def plot_category_count(df, categories):
    sns.set(font_scale = 2)
    plt.figure(figsize=(15,8))

    ax= sns.barplot(categories, df.iloc[:,3:].sum().values)

    plt.title("Jokes in each category", fontsize=24)
    plt.ylabel('Number of jokes', fontsize=18)
    plt.xlabel('Joke Skill', fontsize=18)

    #adding the text labels
    rects = ax.patches
    #print(rects)
    labels = df.iloc[:,3:].sum().values
    #print(labels)
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=18)

    plt.show()

In [12]:
#plot_category_count(df, categories)

### 1.3. Calculating number of jokes having multiple labels

In [13]:
def plot_multiple_label(mlc_labels, multiLabel_counts):
    sns.set(font_scale = 2)
    plt.figure(figsize=(15,8))

    ax = sns.barplot(mlc_labels, multiLabel_counts.values)

    plt.title("Jokes having multiple labels ")
    plt.ylabel('Number of jokes', fontsize=18)
    plt.xlabel('Number of labels', fontsize=18)

    #adding the text labels
    rects = ax.patches
    labels = multiLabel_counts.values
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

    plt.show()

In [14]:
def print_multiple_label(df):
    rowSums = df.iloc[:,3:].sum(axis=1)
    multiLabel_counts = rowSums.value_counts()
    print(multiLabel_counts)
    multiLabel_counts = multiLabel_counts.iloc[:]
    #print(multiLabel_counts.index)
    mlc_labels = ['L'+str(i) for i in multiLabel_counts.index]
    print(mlc_labels)
    
    #plot_multiple_label(mlc_labels, multiLabel_counts)
    ##return(mlc_labels, multiLabel_counts)

In [15]:
print_multiple_label(df)

1    3301
2      57
0       6
3       1
dtype: int64
['L1', 'L2', 'L0', 'L3']


### 1.4. WordCloud representation of most used words in each category of jokes

In [16]:
#!pip install wordcloud

In [17]:
from wordcloud import WordCloud, STOPWORDS
import jieba
import Stopwords

In [18]:
plt.figure(figsize=(40,25))

def MyWordCloud(plt, df, field, position):
    #subset = df[df.Pun==1]
    subset = df.loc[df[field] == 1] # https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values
    #print(subset.head()); exit
    text = str(subset.Content.values)
    words_list = jieba.lcut(Stopwords.clean_text(text))
    text = Stopwords.clean_words(words_list)
    cloud = WordCloud(
                          #stopwords=STOPWORDS,
                          stopwords=Stopwords.STOP_WORDS,
                          background_color='black',
                          font_path='SNsanafonGyou.ttf', # OSError: unknown file format
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))

    plt.subplot(3, 3, position)
    plt.axis('off')
    plt.title(field, fontsize=40)
    plt.imshow(cloud)
    return plt

'''
for i, c in enumerate(categories):
    plt = MyWordCloud(plt, df, c, i+1)

#plt.show()
'''

'\nfor i, c in enumerate(categories):\n    plt = MyWordCloud(plt, df, c, i+1)\n\n#plt.show()\n'

<Figure size 2880x1800 with 0 Axes>

## 2. Data Pre-Processing

In [19]:
# Compute statistics of the dataset: MaxLength, MinLength, AvgChars, AvgWords
Len = df.Content.map(len)
print(f'Number of characters in   all jokes: Max={max(Len)}, Min={min(Len)}, Avg={sum(Len)/len(Len)}')
Len = train_random.Content.map(len)
print(f'Number of characters in train jokes: Max={max(Len)}, Min={min(Len)}, Avg={sum(Len)/len(Len)}')
Len = test.Content.map(len)
print(f'Number of characters in  test jokes: Max={max(Len)}, Min={min(Len)}, Avg={sum(Len)/len(Len)}')

Number of characters in   all jokes: Max=2024, Min=10, Avg=134.07637444279345
Number of characters in train jokes: Max=2024, Min=10, Avg=132.7906564163217
Number of characters in  test jokes: Max=874, Min=12, Avg=135.3751493428913


In [20]:
# set global variables: data
data = df
#data = df.loc[np.random.choice(df.index, size=3365)]
data.shape

(3365, 9)

In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

### 2.1. Cleaning Data

In [22]:
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext

def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

In [23]:
import Stopwords # import my own module with STOP_WORDS
from nltk.stem import PorterStemmer, WordNetLemmatizer
ps = PorterStemmer()
wnl = WordNetLemmatizer()

In [24]:
def clean_text(text): 
    '''
    Given a raw text string, return a clean text string.
    Example: 
        input:  "Years  passed. 多少   年过 去 了 。  "
        output: "years passed.多少年过去了。"
    '''
    text = str(text)
    text = text.lower() # 'years  passed. 多少   年过 去 了 。'
    # Next line will remove redundant white space for jeiba to cut
    text = re.sub(r'\s+([^a-zA-Z0-9.])', r'\1', text) # years passed.多少年过去了。
# see: https://stackoverflow.com/questions/16720541/python-string-replace-regular-expression
    text = text.strip(' ')
    return text

def clean_words(text, RmvStopWord=True, RmvMark=True):
    words = jieba.lcut(text)
#    print("After jieba.lcut():", words)
#    WL = [ w 
    WL = [ ps.stem(w)
#    WL = [ wnl.lemmatize(w)
        for w in words 
          if (not re.match(r'\s', w)) # remove white spaces
            and (RmvMark==False or not re.match(r'\W', w)) # remove punctuations
#            and (RmvMark==False or not re.match('^[a-z_]$', w)) # remove punctuations
#            and (RmvMark==False or w not in PUNCTUATIONS)
            and (RmvStopWord==False or w not in Stopwords.STOP_WORDS)
            and (not re.match(r'^\d+$', w)) # remove digit
         ]
    WL = " ".join(WL)
    return WL

In [25]:
print(data.head())
data['Content'] = data['Content'].str.lower()
#data['Content'] = data['Content'].apply(cleanHtml)
#data['Content'] = data['Content'].apply(cleanPunc)
#data['Content'] = data['Content'].apply(keepAlpha)
data['Content'] = data['Content'].apply(clean_text)
data['Content'] = data['Content'].apply(clean_words)
#data.head()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/kg/jcdj05xn20144cv9kwywp26r0000gn/T/jieba.cache


      ID Title                                            Content  Affinity  \
0  L0001  要求加薪  員工：老闆，您必須幫我加薪，已經有三家公司在找我了！     老闆：哪三家？     員工：...         0   
1  L0002  查無此人  某市政府辦公大樓落成，門口缺副對聯。     副市長揮毫     上聯：說實話辦實事一身正氣...         0   
2  L0003   遣散費  中午老闆視察自己的建築工地時，發現有個人在角落玩手機。     老闆：你月薪多少？     ...         0   
3  L0004  職業習慣  一天，一位法官的女友看見兩個蚊子，便叫法官打死。     只見法官只把那個肚子飽飽的蚊子打死...         0   
4  L0005  美女吵架  辦公室中兩位女同事吵起來了。     經理忍無可忍：「太不像話了！現在是什麼情況？你們把原因...         0   

   Self_Improvement  Attack  Self_Depression  Taboo  Others  
0                 0       0                1      0       0  
1                 0       1                0      0       0  
2                 0       0                1      0       0  
3                 0       0                0      0       1  
4                 0       1                0      0       0  


Loading model cost 0.617 seconds.
Prefix dict has been built succesfully.


### 2.2. Removing Stop Words

In [26]:
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

data['Content'] = data['Content'].apply(removeStopWords)
#data.head()

### 2.3. Stemming

In [27]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

data['Content'] = data['Content'].apply(stemming)
data.head()

,ID,Title,Content,Affinity,Self_Improvement,Attack,Self_Depression,Taboo,Others
0,L0001,要求加薪,員工 老 闆 必須 幫 加薪 已經 三家 公司 找 老 闆 三家 員工 自來 水 公司 台電...,0,0,0,1,0,0
1,L0002,查無此人,某 市政府 辦公大樓 落成 門口 缺 副 對聯 副 市長 揮 毫上 聯 實話 辦實事 一身 ...,0,0,1,0,0,0
2,L0003,遣散費,中午 老 闆 視察 自己 建築 工地 時 發現 個 人 角落 玩手 機 老 闆 月薪 多少 ...,0,0,0,1,0,0
3,L0004,職業習慣,一天 一位 法官 女友 看見 兩個 蚊子 便 叫 法官 打死 只見 法官 只 那個 肚子 飽...,0,0,0,0,0,1
4,L0005,美女吵架,辦 公室 中 兩位 女同事 吵起 經理 忍無可忍 太不像 話 情況 原因 給我 清楚 兩人 ...,0,0,1,0,0,0


### 2.4. Train-Test Split

In [28]:
from sklearn.model_selection import train_test_split

# set global variables: train, test
#train, test = train_test_split(data, random_state=42, test_size=0.30, shuffle=True)
train, test = train_test_split(data, random_state=42, train_size=1691, shuffle=False)

print(train.shape)
print(test.shape)

(1691, 9)
(1674, 9)


In [29]:
# set global variables: train_text, test_text
train_text = train['Content']
test_text = test['Content']

In [30]:
# Compute statistics of the dataset: MaxLength, MinLength, AvgChars, AvgWords
Len = data.Content.map(lambda x: len(x.split()))
print(f'Number of words in   all jokes: Max={max(Len)}, Min={min(Len)}, Avg={sum(Len)/len(Len)}')
Len = train.Content.map(lambda x: len(x.split()))
print(f'Number of words in train jokes: Max={max(Len)}, Min={min(Len)}, Avg={sum(Len)/len(Len)}')
Len = test.Content.map(lambda x: len(x.split()))
print(f'Number of words in  test jokes: Max={max(Len)}, Min={min(Len)}, Avg={sum(Len)/len(Len)}')

Number of words in   all jokes: Max=491, Min=3, Avg=41.6222882615156
Number of words in train jokes: Max=491, Min=3, Avg=40.33234772324069
Number of words in  test jokes: Max=290, Min=4, Avg=42.92532855436081


### 2.5. TF-IDF

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
#vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', 
                             ngram_range=(1,2), norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents='unicode', sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

## Set global variables: x_train, y_train, x_test, y_test

In [33]:
x_train = vectorizer.transform(train_text)
y_train = train_random.drop(labels = ['ID', 'Title', 'Content'], axis=1)

x_test = vectorizer.transform(test_text)
y_test = test.drop(labels = ['ID', 'Title', 'Content'], axis=1)

In [34]:
time_TfidfVector = time.time()

def Create_TFxIDF(data_text, train_text, test_text):

# word level tf-idf
    #tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=10000)
    tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
        stop_words=Stopwords.STOP_WORDS, max_df=0.95, min_df=2, max_features=10000)
    tfidf_vect.fit(data_text)
    xtrain_tfidf = tfidf_vect.transform(train_text)
    xtest_tfidf = tfidf_vect.transform(test_text)
    print(f"xtrain_tfidf.shape:{xtrain_tfidf.shape}, xtest_tfidf.shape: {xtest_tfidf.shape}")

# word level ngram tf-idf 
    tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                    stop_words=Stopwords.STOP_WORDS, max_df=0.95, min_df=2,
                    ngram_range=(2,3), max_features=10000)
    tfidf_vect_ngram.fit(data_text)
    xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_text)
    xtest_tfidf_ngram =  tfidf_vect_ngram.transform(test_text)
    print(f"xtrain_tfidf_ngram.shape:{xtrain_tfidf.shape}, xtest_tfidf_ngram.shape: {xtest_tfidf.shape}")

# character level ngram tf-idf
    tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', 
                    stop_words=Stopwords.STOP_WORDS, max_df=0.95, min_df=2,
                    ngram_range=(2,3), max_features=10000)
    tfidf_vect_ngram_chars.fit(data_text)
    xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_text) 
    xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(test_text) 
    print(f"xtrain_tfidf_ngram_chars.shape:{xtrain_tfidf.shape}, xtest_tfidf_ngram_chars.shape: {xtest_tfidf.shape}")

    print("It takes %4.2f seconds to convert 3 TFxIDF vectors."%(time.time()-time_TfidfVector))

    return (xtrain_tfidf, xtest_tfidf, 
             xtrain_tfidf_ngram, xtest_tfidf_ngram,
             xtrain_tfidf_ngram_chars, xtest_tfidf_ngram_chars,
            tfidf_vect, tfidf_vect_ngram, tfidf_vect_ngram_chars)

(xtrain_tfidf, xtest_tfidf, 
 xtrain_tfidf_ngram, xtest_tfidf_ngram,
 xtrain_tfidf_ngram_chars, xtest_tfidf_ngram_chars,
 tfidf_vect, tfidf_vect_ngram, tfidf_vect_ngram_chars) = Create_TFxIDF(data.Content, train_text, test_text)

xtrain_tfidf.shape:(1691, 9559), xtest_tfidf.shape: (1674, 9559)
xtrain_tfidf_ngram.shape:(1691, 9559), xtest_tfidf_ngram.shape: (1674, 9559)
xtrain_tfidf_ngram_chars.shape:(1691, 9559), xtest_tfidf_ngram_chars.shape: (1674, 9559)
It takes 2.49 seconds to convert 3 TFxIDF vectors.


In [35]:
# re-assign x_train and x_test to what we want
x_train, x_test, vectorizer = xtrain_tfidf, xtest_tfidf, tfidf_vect
#x_train, x_test, vectorizer = xtrain_tfidf_ngram, xtest_tfidf_ngram, tfidf_vect_ngram
#x_train, x_test, vectorizer = xtrain_tfidf_ngram_chars, xtest_tfidf_ngram_chars, tfidf_vect_ngram_chars
print(x_train.shape, x_test.shape)
#print(x_train)

(1691, 9559) (1674, 9559)


## 3. Multi-Label Classification

### 3.1. Multiple Binary Classifications - (One Vs Rest Classifier)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [37]:
# define the evaluation metrics
def tcfunc(x, n=4): # trancate a number to have n decimal digits
    d = '0' * n
    d = int('1' + d)
# https://stackoverflow.com/questions/4541155/check-if-a-number-is-int-or-float
    if isinstance(x, (int, float)): return int(x * d) / d
    return x

def print_cls_report(y_true, prediction):
    print('Test accuracy is %1.4f'%(accuracy_score(y_true, prediction)))

    print(classification_report(y_true, prediction))
    
    # http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
    print("\tPrecision\tRecall\tF1\tSupport")
    (Precision, Recall, F1, Support) = list(map(tcfunc, 
        precision_recall_fscore_support(y_true, prediction, average='micro')))
    print("Micro\t{}\t{}\t{}\t{}".format(Precision, Recall, F1, Support))

    (Precision, Recall, F1, Support) = list(map(tcfunc, 
        precision_recall_fscore_support(y_true, prediction, average='macro')))
    print("Macro\t{}\t{}\t{}\t{}".format(Precision, Recall, F1, Support))
    
#    if True:
    if False:
        print(confusion_matrix(y_true, prediction))
        try: 
            print(classification_report(y_true, prediction, digits=4))
        except ValueError:
            print('May be some category has no predicted samples')
        show_confusion_matrix(prediction)


    print(f'y_true.shape={y_true.shape}, prediction.shape={prediction.shape}')
    #print(y_true.head())
    #print(prediction[0:6])

    # https://stackoverflow.com/questions/152580/whats-the-canonical-way-to-check-for-type-in-python
    pred = prediction
    if not isinstance(pred, np.ndarray): pred = prediction.toarray()
    print(type(y_true), type(prediction), type(pred))
    try:
        # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
        print('macro roc_auc_score is %1.4f'%(roc_auc_score(y_true, pred, average='macro'))) # default average=’macro’
        print('micro roc_auc_score is %1.4f'%(roc_auc_score(y_true, pred, average='micro')))
    except:
        print("roc_auc_score error!!!")
    try:
        fpr, tpr, thresholds = roc_curve(y_true, pred)
        print(f'fpr={fpr}\ntpr={tpr}\nthresholds={thresholds}')
    except:
        print('roc_curve error!!!')

In [38]:
%%time

# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

#for category in categories:
for category in categories:
    printmd('**Processing {} jokes...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train_random[category])
    
    prediction = LogReg_pipeline.predict(x_test)
    
    print_cls_report(test[category], prediction)

**Processing Affinity jokes...**

Test accuracy is 0.9886
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1655
           1       0.00      0.00      0.00        19

   micro avg       0.99      0.99      0.99      1674
   macro avg       0.49      0.50      0.50      1674
weighted avg       0.98      0.99      0.98      1674

	Precision	Recall	F1	Support
Micro	0.9886	0.9886	0.9886	None
Macro	0.4943	0.5	0.4971	None
y_true.shape=(1674,), prediction.shape=(1674,)
<class 'pandas.core.series.Series'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.5000
micro roc_auc_score is 0.5000
fpr=[0. 1.]
tpr=[0. 1.]
thresholds=[1 0]


**Processing Self_Improvement jokes...**

Test accuracy is 0.9904
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1658
           1       0.00      0.00      0.00        16

   micro avg       0.99      0.99      0.99      1674
   macro avg       0.50      0.50      0.50      1674
weighted avg       0.98      0.99      0.99      1674

	Precision	Recall	F1	Support
Micro	0.9904	0.9904	0.9904	None
Macro	0.4952	0.5	0.4975	None
y_true.shape=(1674,), prediction.shape=(1674,)
<class 'pandas.core.series.Series'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.5000
micro roc_auc_score is 0.5000
fpr=[0. 1.]
tpr=[0. 1.]
thresholds=[1 0]


**Processing Attack jokes...**

...

Test accuracy is 0.9898
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1657
           1       0.00      0.00      0.00        17

   micro avg       0.99      0.99      0.99      1674
   macro avg       0.49      0.50      0.50      1674
weighted avg       0.98      0.99      0.98      1674

	Precision	Recall	F1	Support
Micro	0.9898	0.9898	0.9898	None
Macro	0.4949	0.5	0.4974	None
y_true.shape=(1674,), prediction.shape=(1674,)
<class 'pandas.core.series.Series'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.5000
micro roc_auc_score is 0.5000
fpr=[0. 1.]
tpr=[0. 1.]
thresholds=[1 0]


**Processing Taboo jokes...**

Test accuracy is 0.8632
              precision    recall  f1-score   support

           0       0.86      1.00      0.93      1445
           1       0.00      0.00      0.00       229

   micro avg       0.86      0.86      0.86      1674
   macro avg       0.43      0.50      0.46      1674
weighted avg       0.75      0.86      0.80      1674

	Precision	Recall	F1	Support
Micro	0.8632	0.8632	0.8632	None
Macro	0.4316	0.5	0.4632	None
y_true.shape=(1674,), prediction.shape=(1674,)
<class 'pandas.core.series.Series'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.5000
micro roc_auc_score is 0.5000
fpr=[0. 1.]
tpr=[0. 1.]
thresholds=[1 0]


**Processing Others jokes...**

Test accuracy is 0.7037
              precision    recall  f1-score   support

           0       0.23      0.08      0.12       420
           1       0.75      0.91      0.82      1254

   micro avg       0.70      0.70      0.70      1674
   macro avg       0.49      0.50      0.47      1674
weighted avg       0.62      0.70      0.65      1674

	Precision	Recall	F1	Support
Micro	0.7037	0.7037	0.7037	None
Macro	0.4898	0.4958	0.4697	None
y_true.shape=(1674,), prediction.shape=(1674,)
<class 'pandas.core.series.Series'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.4958
micro roc_auc_score is 0.4958
fpr=[0.         0.92142857 1.        ]
tpr=[0.         0.91307815 1.        ]
thresholds=[2 1 0]
CPU times: user 105 ms, sys: 78.3 ms, total: 183 ms
Wall time: 1.77 s


### 3.2. Multiple Binary Classifications - (Binary Relevance)

In [39]:
# To use binary relevance, run "pip install scikit-multilearn" in advance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
# Next line refers to: http://scikit.ml/tutorial.html
from sklearn.svm import SVC, LinearSVC

In [40]:
%%time

# initialize binary relevance multi-label classifier
#   with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

printmd('**BinaryRelevance with GaussianNB()**')
print_cls_report(y_test, predictions)

**BinaryRelevance with GaussianNB()**

Test accuracy is 0.4815
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        16
           2       0.05      0.03      0.04       143
           3       0.00      0.00      0.00        17
           4       0.11      0.07      0.09       229
           5       0.74      0.65      0.69      1254

   micro avg       0.62      0.50      0.55      1678
   macro avg       0.15      0.13      0.14      1678
weighted avg       0.57      0.50      0.53      1678
 samples avg       0.49      0.50      0.49      1678

	Precision	Recall	F1	Support
Micro	0.6156	0.4982	0.5507	None
Macro	0.1508	0.1263	0.1371	None
y_true.shape=(1674, 6), prediction.shape=(1674, 6)
<class 'pandas.core.frame.DataFrame'> <class 'scipy.sparse.csc.csc_matrix'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.4934
micro roc_auc_score is 0.7179
roc_curve error!!!
CPU times: user 10.3 s, sys: 780 ms, total: 11.1 s
Wall 

In [41]:
%%time

# Next line refers to: http://scikit.ml/tutorial.html and 
#   http://scikit.ml/api/skmultilearn.problem_transform.br.html
#classifier = BinaryRelevance(classifier=SVC(), require_dense=[False, True]) # 0.5 very bad!
# https://scikit-learn.org/stable/modules/svm.html#unbalanced-problems
classifier = BinaryRelevance(classifier=LinearSVC(class_weight='balanced')) # 0.5314
#classifier = BinaryRelevance(classifier=LinearSVC()) # Test roc_auc_score is 0.5234

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

printmd('**BinaryRelevance with LinearSVC()**')
print_cls_report(y_test, predictions)

**BinaryRelevance with LinearSVC()**

Test accuracy is 0.4277
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        16
           2       0.02      0.01      0.02       143
           3       0.00      0.00      0.00        17
           4       0.10      0.07      0.08       229
           5       0.74      0.61      0.67      1254

   micro avg       0.59      0.46      0.52      1678
   macro avg       0.14      0.12      0.13      1678
weighted avg       0.57      0.46      0.51      1678
 samples avg       0.45      0.47      0.45      1678

	Precision	Recall	F1	Support
Micro	0.5905	0.4642	0.5198	None
Macro	0.1425	0.1157	0.1277	None
y_true.shape=(1674, 6), prediction.shape=(1674, 6)
<class 'pandas.core.frame.DataFrame'> <class 'scipy.sparse.csc.csc_matrix'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.4891
micro roc_auc_score is 0.6998
roc_curve error!!!
CPU times: user 1.27 s, sys: 175 ms, total: 1.44 s
Wall 

### 3.3. Classifier Chains

In [42]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

In [43]:
%%time

# initialize classifier chains multi-label classifier
#classifier = ClassifierChain(LogisticRegression()) # Test roc_auc_score is 0.5159
classifier = ClassifierChain(LinearSVC(class_weight='balanced')) #  0.5327

# Training logistic regression model on train data
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

printmd('**Classifier Chains with LinearSVM()**')
print_cls_report(y_test, predictions)

**Classifier Chains with LinearSVM()**

Test accuracy is 0.5597
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        16
           2       0.03      0.02      0.02       143
           3       0.00      0.00      0.00        17
           4       0.10      0.14      0.12       229
           5       0.73      0.72      0.73      1254

   micro avg       0.56      0.56      0.56      1678
   macro avg       0.14      0.15      0.14      1678
weighted avg       0.57      0.56      0.56      1678
 samples avg       0.56      0.56      0.56      1678

	Precision	Recall	F1	Support
Micro	0.5609	0.5595	0.5602	None
Macro	0.1438	0.1469	0.1448	None
y_true.shape=(1674, 6), prediction.shape=(1674, 6)
<class 'pandas.core.frame.DataFrame'> <class 'scipy.sparse.csc.csc_matrix'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.4855
micro roc_auc_score is 0.7359
roc_curve error!!!
CPU times: user 2.43 s, sys: 738 ms, total: 3.17 s
Wall 

### 3.4. Label Powerset

In [44]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset

In [45]:
%%time

# initialize label powerset multi-label classifier
#classifier = LabelPowerset(LogisticRegression()) # Test roc_auc_score is 0.5059
classifier = LabelPowerset(LinearSVC(class_weight='balanced')) # 0.5541
# Test roc_auc_score is 0.5312 if xtrain_tfidf_ngram, xtest_tfidf_ngram are used.
# Test roc_auc_score is 0.5474 if xtrain_tfidf_ngram_chars, xtest_tfidf_ngram_chars are used

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

printmd('**Label Powerset with LinearSVM()**')
print_cls_report(y_test, predictions)

**Label Powerset with LinearSVM()**

Test accuracy is 0.5233
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        16
           2       0.04      0.04      0.04       143
           3       0.00      0.00      0.00        17
           4       0.09      0.09      0.09       229
           5       0.74      0.69      0.71      1254

   micro avg       0.53      0.53      0.53      1678
   macro avg       0.14      0.14      0.14      1678
weighted avg       0.57      0.53      0.55      1678
 samples avg       0.53      0.53      0.53      1678

	Precision	Recall	F1	Support
Micro	0.5257	0.5297	0.5277	None
Macro	0.1448	0.1368	0.1406	None
y_true.shape=(1674, 6), prediction.shape=(1674, 6)
<class 'pandas.core.frame.DataFrame'> <class 'scipy.sparse.lil.lil_matrix'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.4813
micro roc_auc_score is 0.7170
roc_curve error!!!
CPU times: user 876 ms, sys: 61.8 ms, total: 938 ms
Wall

In [46]:
with open('out/Skill_True.txt', 'w') as outF:
    outF.write(y_test.to_csv(sep='\t', index=False))

# https://stackoverflow.com/questions/36967666/transform-scipy-sparse-csr-to-pandas
with open('out/Skill_Pred_random.txt', 'w') as outF:
    outF.write(pd.DataFrame(predictions.toarray(), columns=list(y_test.columns)).to_csv(sep='\t', index=False))

### 3.5. Adapted Algorithm

In [47]:
# http://scikit.ml/api/api/skmultilearn.adapt.html#skmultilearn.adapt.MLkNN
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

In [48]:
%%time

classifier_new = MLkNN(k=10)

# Note that this classifier can throw up errors when handling sparse matrices.

x_train = lil_matrix(x_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(x_test).toarray()

# train
classifier_new.fit(x_train, y_train)

# predict
predictions_new = classifier_new.predict(x_test)

printmd('**MLkNN**')
print_cls_report(y_test, predictions_new)

**MLkNN**

Test accuracy is 0.4719
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        16
           2       0.00      0.00      0.00       143
           3       0.00      0.00      0.00        17
           4       0.00      0.00      0.00       229
           5       0.75      0.63      0.69      1254

   micro avg       0.74      0.47      0.58      1678
   macro avg       0.12      0.11      0.11      1678
weighted avg       0.56      0.47      0.51      1678
 samples avg       0.47      0.47      0.47      1678

	Precision	Recall	F1	Support
Micro	0.742	0.4713	0.5765	None
Macro	0.1249	0.1051	0.1141	None
y_true.shape=(1674, 6), prediction.shape=(1674, 6)
<class 'pandas.core.frame.DataFrame'> <class 'scipy.sparse.lil.lil_matrix'> <class 'numpy.ndarray'>
macro roc_auc_score is 0.4996
micro roc_auc_score is 0.7193
roc_curve error!!!
CPU times: user 1min 10s, sys: 323 ms, total: 1min 11s
Wa